In [5]:
from __future__ import annotations

import os
from dataclasses import dataclass
from typing import TypedDict, Dict, Sequence, Union, Optional, Any
import asyncio

from asgiref.sync import sync_to_async
from langchain_core.prompts import PromptTemplate, SystemMessagePromptTemplate,HumanMessagePromptTemplate,ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from langchain.chat_models import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph.message import add_messages
from fastapi_server.agent.prompt import (
    document_type_system_prompt_agent2,
    proceedings_summary_prompt_agent2,
    internal_policy_summary_prompt_template_agent2,
    product_document_summary_prompt_template_agent2,
    technical_document_summary_prompt_template_agent2,
    unknown_document_type_prompt_agent2,
    rag_answer_generation_prompt_agent2,
    rag_system_message_agent2,
    create_similar_questions_agent2
)
load_dotenv()

def init_clients():
    # 1-1) OpenAI 클라이언트 생성
    openai_api_key = os.getenv("OPENAI_API_KEY")
    if not openai_api_key:
        raise ValueError("⚠️ 환경변수 OPENAI_API_KEY가 설정되지 않았습니다.")
    # OpenAI 인스턴스를 만듭니다.
    openai_client = OpenAI(api_key=openai_api_key)
    print("✅ OpenAI 클라이언트 생성 완료")

    # 1-2) Pinecone 인스턴스 생성
    pinecone_api_key = os.getenv("PINECONE_API_KEY")
    pinecone_env     = os.getenv("PINECONE_ENVIRONMENT")   # 예: "us-east1-gcp" 또는 "us-west1-gcp" 등
    if not pinecone_api_key or not pinecone_env:
        raise ValueError("⚠️ 환경변수 PINECONE_API_KEY 또는 PINECONE_ENVIRONMENT가 누락되었습니다.")

    pc = Pinecone(api_key=pinecone_api_key, environment=pinecone_env)
    print("✅ Pinecone 클라이언트 생성 완료")

    # 1-3) 인덱스 존재 여부 확인
    index_name = "dense-index"  # 실제 사용 중인 인덱스 이름으로 교체하세요
    existing_indexes = pc.list_indexes().names()
    if index_name not in existing_indexes:
        raise ValueError(f"⚠️ 인덱스 '{index_name}'가 Pinecone에 존재하지 않습니다. 현재 인덱스 목록: {existing_indexes}")

    # 1-4) 해당 인덱스 객체 가져오기
    index = pc.Index(index_name)
    print(f"✅ Pinecone 인덱스 '{index_name}' 연결 완료 (Namespaces: {len(index.describe_index_stats().namespaces)})")

    return openai_client, index


# --------------------------------------------------
# 2) 질문 문장을 임베딩 벡터로 변환
# --------------------------------------------------
def embed_query(openai_client: OpenAI, text: str) -> list:
    """
    최신 OpenAI 클라이언트에서는 resp.data[0].embedding 으로 벡터에 접근해야 합니다.
    """
    resp = openai_client.embeddings.create(
        model="text-embedding-3-large",
        input=text
    )
    return resp.data[0].embedding

# --------------------------------------------------
# 3) 검색된 매칭 결과에서 실제 텍스트(메타데이터)를 꺼내 Context 로 결합
# --------------------------------------------------
def build_context_from_matches(matches):
    """
    res.matches 리스트 안의 각 item.metadata 에 들어 있는 텍스트 필드를 추출합니다.
    업로드 시 metadata 키가 "text"였다고 가정했습니다.
    """
    contexts = []
    for m in matches:
        chunk_text = m.metadata.get("text", "")
        if chunk_text:
            contexts.append(chunk_text)

    return "\n---\n".join(contexts)

# --------------------------------------------------
# 4) fetch_res.vectors에서 metadata중에 text만 추출해서 join
# --------------------------------------------------
def combine_text(fetch_res, final_doc_ids) :
    texts = []
    for doc_id in final_doc_ids:
        vec_info = fetch_res.vectors.get(doc_id)
        text = vec_info["metadata"].get("text", "")
        if text:
            texts.append(text)

    # 4) 최종 context 조립
    context = "\n---\n".join(texts)
    return context

# --------------------------------------------------
# 5) 검색된 매칭 결과에서 문서 ID 추출
# --------------------------------------------------
def get_document_id(matches) :
    ids = []
    for m in matches:
        ids.append(m.id)
        print(m.score,end=" ") # 디버깅용 점수 출력 (pinecone은 기본적으로 점수 내림차순으로 matches가 정렬됨)
    print() 
    return ids


# --------------------------------------------------
# 6) 유사 질문 생성 함수 (4개의 유사 질문 생성)
# --------------------------------------------------
def create_similar_questions(message) :
    client = OpenAI()
    formatted_prompt = create_similar_questions_agent2.format(user_input=message)
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": formatted_prompt}
        ],
        temperature=0,
        max_tokens=100
    )
    
    similar_questions = resp.choices[0].message.content.strip().split("\n")
    print(similar_questions)
    print(f"생성된 유사 질문의 자료형 : {type(similar_questions)}")
    return similar_questions


# 1) openai_client와 index 초기화
openai_client, pinecone_index = init_clients()
namespace_to_search = "internal_policy"
index_stats = pinecone_index.describe_index_stats()

# 2) RAG Fusion을 위한 dictionary 초기화
# question_to_doc : 질문번호와 해당 질문에 대한 문서 ID 매핑
# document_score : 문서 id와 해당 문서의 RRF 점수 매핑
question_to_doc = dict.fromkeys([0,1,2,3,4])
document_score = dict()

# 3) 질문을 유사 질문 4개로 확장하고, 원본 질문도 포함
# similar_questions : 유사 질문 4개 + 원본 질문 1개
similar_questions = create_similar_questions("휴가는 최대 몇일까지 사용 가능한가요?")
similar_questions.append("휴가는 최대 몇일까지 사용 가능한가요?") 
    
# 4) RAG Fusion을 위한 질문-문서 매핑
# 각 질문마다 embedding 벡터를 생성하고 Pinecone에서 유사도 검색을 수행하여 최대 문서 4개를 찾는다.
# 각 질문에 대해 문서 ID를 매핑함.
for i , question in enumerate(similar_questions) :
    query_vector = embed_query(openai_client, question.strip())
    if not namespace_to_search or namespace_to_search == "unknown":
        message = f"문서 타입이 '{namespace_to_search}'(으)로 분류되어 Pinecone 검색을 수행하지 않습니다."
        # 이 코드는 execute_rag_node가 'unknown' 타입으로 호출될 경우를 대비한 방어 코드입니다.

    if namespace_to_search not in index_stats.namespaces or \
        index_stats.namespaces[namespace_to_search].vector_count == 0:
        message = f"'{namespace_to_search}' 네임스페이스를 Pinecone에서 찾을 수 없거나, 해당 네임스페이스에 \
        데이터가 없습니다. Pinecone 대시보드에서 네임스페이스 이름과 데이터 존재 여부를 확인해주세요."

    res = pinecone_index.query(
        vector=query_vector,
        namespace=namespace_to_search,
        top_k=4, # 검색할 문서 수
        include_metadata=True
    )

    matches = res.matches
    print(f"질문 {i+1} : '{question}'")
    print(f"   - Pinecone 검색 완료: {len(matches)}개 결과 수신")

    if not matches:
        message = f"'{namespace_to_search}' 네임스페이스에서 '{question}' 질문과 관련된 정보를 찾지 못했습니다."
        # print(f"   - 정보 없음: {message}")
    question_to_doc[i] = get_document_id(matches)

print(question_to_doc) # 디버깅용 출력 (질문1~5에 대한 문서 id 매핑)
print()

# 5) RRF 점수 계산
for key in question_to_doc :
    for i, doc_id in enumerate(question_to_doc[key]) :
        if doc_id not in document_score :
            document_score[doc_id] = float(1/(60+1+i))
        else : 
            document_score[doc_id] += float(1/(60+1+i))
print(f"문서 점수 : {document_score}") # 디버깅용 출력 (문서 id와 점수 매핑)

# 6) RRF 점수가 높은 상위 2개 슬라이싱
top2 = sorted(
    document_score.items(),
    key=lambda x: x[1],
    reverse=True
)[:2]
final_doc_ids = [doc_id for doc_id, _ in top2]
print(final_doc_ids) # 디버깅용 출력 (최종 문서 ID 리스트)

# 7) 최종 문서 ID를 사용하여 Pinecone에서 fetch
fetch_res = pinecone_index.fetch(
    ids=final_doc_ids,
    namespace=namespace_to_search
)

# 8) 최종 context 조립
context = combine_text(fetch_res, final_doc_ids)
print(f"(길이: {len(context)})")  
import pprint
print(f"컨텍스트: {pprint.pprint(context)}\n")
    



✅ OpenAI 클라이언트 생성 완료
✅ Pinecone 클라이언트 생성 완료
✅ Pinecone 인덱스 'dense-index' 연결 완료 (Namespaces: 4)
['휴가는 언제까지 사용할 수 있나요?  ', '휴가의 유효 기간은 언제까지인가요?  ', '남은 휴가는 최대 몇 일까지 사용할 수 있나요?  ', '휴가를 사용할 수 있는 마지막 날짜는 언제인가요?']
생성된 유사 질문의 자료형 : <class 'list'>
질문 1 : '휴가는 언제까지 사용할 수 있나요?  '
   - Pinecone 검색 완료: 4개 결과 수신
0.513351202 0.3539235 0.353207737 0.303562015 
질문 2 : '휴가의 유효 기간은 언제까지인가요?  '
   - Pinecone 검색 완료: 4개 결과 수신
0.439063877 0.326840222 0.286609083 0.276723266 
질문 3 : '남은 휴가는 최대 몇 일까지 사용할 수 있나요?  '
   - Pinecone 검색 완료: 4개 결과 수신
0.456431359 0.331084609 0.304434299 0.270802051 
질문 4 : '휴가를 사용할 수 있는 마지막 날짜는 언제인가요?'
   - Pinecone 검색 완료: 4개 결과 수신
0.4537687 0.288793385 0.282272577 0.256568 
질문 5 : '휴가는 최대 몇일까지 사용 가능한가요?'
   - Pinecone 검색 완료: 4개 결과 수신
0.470805794 0.3305282 0.330489516 0.26877892 
{0: ['doc_d5219f14cf1f3aa1c2d0b411f14c4942', 'doc_a654b7d9649ef296468872fb6411afe1', 'doc_ec65497811b9693e6085e295c009a937', 'doc_3ee07f70b8bf82607c7382eae5c8cb75'], 1: ['doc_d5219f14cf1f3aa1c2d0b411f14c494